# R2T1

## reading data

In [1]:
using SparseArrays
using JuMP
using GLPK
using MAT

In [7]:
file = matopen("R2T1.mat")
varnames = names(file)

3-element Array{String,1}:
 "S"
 "l1"
 "u1"

In [8]:
S = read(file, "S")
l1 = read(file, "l1")
u1 = read(file, "u1")

Dict{String,Any} with 3 entries:
  "nzval" => [0.0945981, 0.0962594, 0.00953525, 0.352967, 1.61011, 1.45645, 0.9…
  "nzind" => [2, 3, 4, 10, 14, 15, 17, 21, 27, 36  …  77, 80, 83, 84, 85, 87, 8…
  "n"     => 95

In [9]:
l1_nzval = l1["nzval"]
l1_nzind = l1["nzind"]
l1_n = l1["n"]
l1 = SparseVector(l1_n, l1_nzind, l1_nzval);

In [10]:
u1_nzval = u1["nzval"]
u1_nzind = u1["nzind"]
u1_n = u1["n"]
u1 = SparseVector(u1_n, u1_nzind, u1_nzval);

In [12]:
(m,n)=size(S)
mzeros = zeros(m,1)
nones = ones(n,1);

# 1. zero constraint Idea

## constructing model

In [26]:
model = Model(GLPK.Optimizer)
@variable(model, l1[i] <= v[i = 1:n] <= u1[i])
@variable(model,w[1:n])
@constraint(model, S*v .== mzeros)
@constraint(model,w .>= v)
@constraint(model,w .>= -v)
@objective(model,Min,sum(nones[i]*w[i] for i in 1:n)) #
optimize!(model)
x= JuMP.value.(w)

13094-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  1.124906401398651e-16
  0.0
 -6.961629041525816e-17
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [27]:
a= 1
function get_step(index_zeros,x)
    grad= [exp(-a*x[j]) for j in 1:n]
    model = Model(GLPK.Optimizer)
    @variable(model, l1[i] <= v[i = 1:n] <= u1[i])
    @variable(model,w[1:n])
    @constraint(model, S*v .== mzeros)
    @constraint(model,w .>= v)
    @constraint(model,w .>= -v)
    for i in index_zeros
        @constraint(model,w[i]==0)
    end
    @objective(model,Min,sum(grad[i]*w[i] for i=1:n))
    optimize!(model)
    if sum(grad.*JuMP.value.(w))>=sum(grad.*x)
        return -1, -1
    else    
        return JuMP.value.(w), JuMP.value.(v)
    end    
end 

get_step (generic function with 1 method)

In [28]:
num= 0
answer= []
for i in 1:10
    index_zeros= zeros(Int64 ,n,1)
    k=1
    for j in 1:n
        if x[j]==0
            index_zeros[k]= j
            k= k+1
        end
    end
    index_zeros= index_zeros[1:k-1]
    next_x, next_answer= get_step(index_zeros,x)
    if next_x[1]==-1
        break
    end
    x= next_x
    answer= next_answer
    num+=1
    print(num,'\n')
end    

1
2
3
4
5


In [29]:
function num_zeros(v)
    zero_num= 0
    for i in 1:n
        if abs(v[i])<=1e-10
            zero_num+=1
        end
    end
    return zero_num
end

num_zeros (generic function with 1 method)

In [31]:
#answer= JuMP.value.(v)
zero_num= num_zeros(answer)
must_be_zeros= zeros(Int64, zero_num+1, 1)
others= zeros(n-zero_num, 1)
k=1
ind= 1
for i in 1:n
    if abs(answer[i])<=1e-10
        must_be_zeros[k]= i
        k= k+1
    else
        others[ind]=i
        ind= ind+1
    end
end
print(others)

[1979.0; 1980.0; 2315.0; 2365.0; 2876.0; 4271.0; 4713.0; 4714.0; 5210.0; 5211.0; 5268.0; 5355.0; 5409.0; 5411.0; 5449.0; 5450.0; 5468.0; 5491.0; 5515.0; 5516.0; 5526.0; 5531.0; 5539.0; 5540.0; 5550.0; 5560.0; 5563.0; 5565.0; 5578.0; 5589.0; 5603.0; 5626.0; 5635.0; 5641.0; 5643.0; 5651.0; 5652.0; 5657.0; 5662.0; 5692.0; 5696.0; 5707.0; 5718.0; 5736.0; 5748.0; 5750.0; 5778.0; 5786.0; 5797.0; 5798.0; 5802.0; 5815.0; 5831.0; 5836.0; 5844.0; 5875.0; 5921.0; 5927.0; 5931.0; 5938.0; 5942.0; 5946.0; 5950.0; 5984.0; 5986.0; 5990.0; 5997.0; 6004.0; 6031.0; 6061.0; 6066.0; 6075.0; 6076.0; 6086.0; 6092.0; 6102.0; 6116.0; 6126.0; 6145.0; 6240.0; 6306.0; 6313.0; 6344.0; 6348.0; 6544.0; 6650.0; 8143.0; 8189.0; 8239.0; 8267.0; 8282.0; 9634.0; 9646.0; 9670.0; 9676.0; 9864.0; 9875.0; 9879.0; 9888.0; 10001.0; 11205.0; 11318.0; 11458.0; 11504.0; 11532.0; 11571.0; 12301.0; 12303.0; 12304.0; 12309.0]

In [32]:
using MathOptInterface
tedad= 0
for m in others
    must_be_zeros[length(must_be_zeros)]= convert(Int64, m)
    model = Model(GLPK.Optimizer)
    @variable(model, l1[i] <= v[i = 1:n] <= u1[i])
    @variable(model,w[1:n])
    @constraint(model, S*v .== mzeros)
    @constraint(model,w .>= v)
    @constraint(model,w .>= -v)
    for i in must_be_zeros
        @constraint(model,w[i]==0)
    end
    @objective(model,Min,sum(nones[i]*w[i] for i=1:n)) #
    optimize!(model)
    if termination_status(model)!=MathOptInterface.INFEASIBLE
        answer= JuMP.value.(v)
        print(m)
        break
    end 
    tedad+=1
    if tedad%10==0
        print(tedad)
    end    
end

105411.0

In [148]:
@show termination_status(model)
@show primal_status(model)
@show dual_status(model)
@show objective_value(model)
@show JuMP.value.(v)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 71.91033404910509
JuMP.value.(v) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.304058203180764e-16, -0.17078248193164144, -6.961629041525816e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0304187042958248, 0.0, 0.0, 9.619171167839755e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.39

13094-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  5.304058203180764e-16
 -0.17078248193164144
 -6.961629041525816e-17
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

## writing to output

In [150]:
print(num_zeros(JuMP.value.(v)))
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, JuMP.value.(v))
end

12813

# 2. weight update idea

In [ ]:
function update_weights(w,v,delta)
    n = length(w)
    phi = zeros(n,1)
    for i=1:n
        if(abs(delta+v[i])>10^(-4))
            phi[i] = delta/(delta+v[i])
        else
            phi[i]=w[i]
        end
    end
    return phi
end

In [ ]:
function opt(w,S,l,u)
    model = Model(GLPK.Optimizer)
    @variable(model, l[i] <= v[i = 1:n] <= u[i])
    @variable(model,a[1:n])
    @constraint(model, S*v .== mzeros)
    for i=1:n
        @constraint(model, a[i] >= v[i])
        @constraint(model, a[i] .>= -v[i])
    end
    @objective(model,Min,sum(w[i]*a[i] for i=1:n));
    
    optimize!(model);
    
    return JuMP.value.(v)
end

In [ ]:
v0 = opt(w,S,l,u)
w = ones(n,1)
ww = w
vv = v0;

In [ ]:
for i in 1:5
    ww=update_weights(ww,vv,0.001)
    vv = opt(ww,S,l,u)
    print(num_zeros(vv),"\n")
end

In [ ]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, vv)
end